In [18]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd

In [19]:
# 將人口和面積的原始資料讀入
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

sql_location = "SELECT * FROM raw_location"
sql_population = "SELECT * FROM raw_population"

df_loc = pd.read_sql(sql_location, engine)
df_popu = pd.read_sql(sql_population, engine)

print("資料讀取完成")

資料讀取完成


In [ ]:
# 重新命名人口資料欄位，並清除地區名的空白
columns = ["city", "district", "population"]
df_popu.columns = columns

df_popu["district"] = df_popu["district"].str.replace("　","")


In [ ]:
# 將兩個dataframe按照"市"及"區"欄位結合，並把無人口的地區刪除
df_main = df_loc.merge(df_popu, how='left', on=["city", "district"])

df_main.dropna(inplace=True)

df_6city = df_main.copy()

# 設定六都名及對應的英文
city_dict = {
    "新北市":"NTP",
    "臺北市":"TPE",
    "桃園市":"TYN",
    "臺中市":"TCH",
    "臺南市":"TNA",
    "高雄市":"KSH"
}

In [25]:
# 增加id欄位並編號
df_6city["code"] = df_6city["city"].map(city_dict)

df_6city["seq"] = df_6city.groupby("code").cumcount() + 1

df_6city["seq"] = df_6city["seq"].apply(lambda x: f"{x:03d}")

df_6city["loc_id"] = df_6city["code"] + df_6city["seq"]

df_6city.drop(columns=["code", "seq"], axis=1, inplace=True)

In [26]:
# 重新調整欄位順序
new_col = ['loc_id', 'city', 'district', 'area(km^2)', 'population']

df_6city = df_6city[new_col]

rename_col = ['loc_id', 'city', 'district', 'area', 'population']

df_6city.columns = rename_col

In [ ]:
# 存檔地端
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\loc_population"
file_6city = "loc_with_population_6city.csv"
path_6city = os.path.join(folder, file_6city)

df_6city.to_csv(path_6city, index=False)

In [28]:
# 輸入資料庫
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

df_6city.to_sql(name="location", con=engine, index=False, if_exists="replace")

158